In [4]:
!pip install opencv-python mediapipe-silicon sklearn matplotlib


In [2]:
import cv2
import numpy as np
import os
# from matplotlib import pyplot as plt
import mediapipe as mp
import time

objc[71050]: Class CaptureDelegate is implemented in both /Users/yaju/anaconda3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1577ce480) and /Users/yaju/anaconda3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1326a0860). One of the two will be used. Which one is undefined.
objc[71050]: Class CVWindow is implemented in both /Users/yaju/anaconda3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1577ce4d0) and /Users/yaju/anaconda3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1215c8a68). One of the two will be used. Which one is undefined.
objc[71050]: Class CVView is implemented in both /Users/yaju/anaconda3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1577ce4f8) and /Users/yaju/anaconda3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1215c8a90). One of the two will be used. Which one is undefined.
objc[71050]: Class CVSlider is implemented in both /Users/yaju/anaconda3/lib/python3.9/si

In [3]:
mp_holistic= mp.solutions.holistic      #Detection model
mp_drawing= mp.solutions.drawing_utils  #Drawing connections

In [4]:
def mediapipe_detection(image, model):
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   #Color conversion
    image.flags.writeable= False                    #Image no longer writeable
    results= model.process(image)                   #Make prediction  
    image.flags.writeable= True                     #Image now writeable
    image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)   #Color conversion 
    return image, results

In [7]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)      #Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)       #Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)  #Draw left hand Connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS) #Draw right hand Connections

In [8]:
# cap = cv2.VideoCapture(0)

# #Access Mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():
        
#         #read feed from cam(0)
#         ret, frame= cap.read()
        
#         #make detection
#         image, results = mediapipe_detection(frame, holistic)
        
#         #Draw connections on the frame
#         draw_landmarks(image, results)
        
#         #Output connection to the screen
#         cv2.imshow('OpenCV Feed', image)
        
#         #Exit Program with 'q'
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
# cap.release()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

-1

In [6]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [6]:
#Path for exported data, numpy arrays
DATA_PATH= os.path.join('MP_DATA')

#Actions to detect
# words = np.array(['hi','thanks','iloveyou', 'my', 'name', 'namaste'])
alphabets= np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])
# numbers = np.arange(1,11)

# actions=np.concatenate([words,alphabets,numbers])
actions=alphabets[:5]


#Thirty videos worth of data
no_sequences= 30

#Video length in frames
sequence_length= 30


In [7]:
#Make folders for data collections
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [13]:
# cap = cv2.VideoCapture(0)

# #Access Mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     #Loop through actions
#     for action in actions:
#         #Loop through videos
#         for sequence in range(no_sequences):
#             #Loop through frames  
#             for frame_num in range(sequence_length):
#                  #read feed from cam(0)
#                  ret, frame= cap.read()
                
#                  #make detection
#                  image, results = mediapipe_detection(frame, holistic)
                
#                  #Draw connections on the frame
#                  draw_landmarks(image, results)
                 
#                  #Apply wait logic:
#                  if frame_num== 0:
#                      cv2.putText(image, 'STARTING COLLECTION', (120, 200),
#                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (132, 63, 195), 4, cv2.LINE_AA)
#                      cv2.putText(image, 'Collecting frames for {} Video Number{}'.format(action, sequence), (15, 50),
#                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (130, 99, 0), 4, cv2.LINE_AA)
#                      cv2.waitKey(1000)
#                  else:
#                     cv2.putText(image, 'Collecting frames for {} Video Number{}'.format(action, sequence), (15, 50),
#                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (130, 99, 0), 4, cv2.LINE_AA)
                     
#                  #New Export keypoints
#                  keypoints= extract_keypoints(results)
#                  npy_path= os.path.join(DATA_PATH, action,str(sequence),str(frame_num))
#                  np.save(npy_path, keypoints)
                
#                  #Output connection to the screen
#                  cv2.imshow('OpenCV Feed', image)
                
#                  #Exit Program with 'q'
#                  if cv2.waitKey(10) & 0xFF == ord('q'):
#                      break 
#                  if cv2.waitKey(10) & 0xFF == ord('p'):
#                      cv2.putText(image, 'PAUSE', (120, 200),
#                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (132, 63, 195), 4, cv2.LINE_AA)
#                      cv2.waitKey(120000) 
        
       
# cap.release()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


-1

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical       #if on silicon mac make a miniforge environment for tensorflow

label_map={label:num for num,label in enumerate(actions)}

In [9]:
sequences,labels=[], []
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])     

In [10]:
X= np.array(sequences)
y= to_categorical(labels).astype(int)

In [11]:
X_train,X_test, y_train, y_test=train_test_split(X,y, test_size=0.05)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [13]:
log_dir = os.path.join('Logs')
tb_callback=TensorBoard(log_dir=log_dir)


2022-07-08 23:02:14.239470: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-07-08 23:02:14.239482: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-07-08 23:02:14.239753: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [14]:
model =Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-08 23:02:23.034750: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-08 23:02:23.035000: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [18]:
model.fit(X_train,y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
 1/24 [>.............................] - ETA: 1:26 - loss: 3.2634 - categorical_accuracy: 0.0312

2022-07-08 23:26:54.911682: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-07-08 23:26:54.911703: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


 2/24 [=>............................] - ETA: 1:27 - loss: 3.2620 - categorical_accuracy: 0.0156

2022-07-08 23:26:58.487461: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-07-08 23:26:58.594381: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-07-08 23:26:58.635430: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: Logs/train/plugins/profile/2022_07_08_23_26_58
2022-07-08 23:26:58.691537: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to Logs/train/plugins/profile/2022_07_08_23_26_58/Yajus-MacBook-Pro.local.trace.json.gz
2022-07-08 23:26:58.725219: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: Logs/train/plugins/profile/2022_07_08_23_26_58
2022-07-08 23:26:58.729014: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to Logs/train/plugins/profile/2022_07_08_23_26_58/Yajus-MacBook-Pro.local.memory_profile.json.gz
2022-07-0

24/24 [==============================] - 87s 4s/step - loss: 3.2585 - categorical_accuracy: 0.0364
Epoch 2/2000
24/24 [==============================] - 86s 4s/step - loss: 3.2584 - categorical_accuracy: 0.0351
Epoch 3/2000
24/24 [==============================] - 87s 4s/step - loss: 3.2584 - categorical_accuracy: 0.0364
Epoch 4/2000
24/24 [==============================] - 85s 4s/step - loss: 3.2583 - categorical_accuracy: 0.0364
Epoch 5/2000
24/24 [==============================] - 67s 3s/step - loss: 3.2584 - categorical_accuracy: 0.0364
Epoch 6/2000
24/24 [==============================] - 64s 3s/step - loss: 3.2583 - categorical_accuracy: 0.0364
Epoch 7/2000
24/24 [==============================] - 66s 3s/step - loss: 3.2582 - categorical_accuracy: 0.0364
Epoch 8/2000
24/24 [==============================] - 64s 3s/step - loss: 3.2582 - categorical_accuracy: 0.0351
Epoch 9/2000
24/24 [==============================] - 64s 3s/step - loss: 3.2582 - categorical_accuracy: 0.0364
Epoch

KeyboardInterrupt: 

In [234]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_57 (LSTM)               (None, 30, 64)            442112    
_________________________________________________________________
lstm_58 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_59 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_55 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_56 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_57 (Dense)             (None, 6)                 198       
Total params: 596,774
Trainable params: 596,774
Non-trainable params: 0
_______________________________________________

In [244]:
res=model.predict(X_test)
actions[np.argmax(res[2])]

'iloveyou'

In [245]:
actions[np.argmax(y_test[2])]

'iloveyou'

In [6]:
from keras.models import load_model
model= load_model('words.h5')

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-29 23:21:47.211384: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-29 23:21:47.211953: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
sequence=[]
sentence=[]
threshold=0.4

cap = cv2.VideoCapture(0)

#Access Mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        #read feed from cam(0)
        ret, frame= cap.read()
        
        #make detection
        image, results = mediapipe_detection(frame, holistic)
        
        #Draw connections on the frame
        draw_landmarks(image, results)
        
        #1.prediction Logic
        keypoints= extract_keypoints(results)
        sequence.append(keypoints)
        sequence= sequence[-30:]
        
        if len(sequence)== 30:
            res= model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
        
        #Output connection to the screen
        cv2.imshow('OpenCV Feed', image)
        
        #Exit Program with 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


NameError: name 'model' is not defined

: 